In [ ]:
import cv2
import numpy as np

# New section

In [ ]:
import cv2

# Load the video
input_video_path = "sample.mp4"
output_video_path = "sample_output.mp4"

# Load the pre-trained face detector (Haar Cascade)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Open the input video
video = cv2.VideoCapture(input_video_path)

# Get the frame width, height, and FPS
frame_width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(video.get(cv2.CAP_PROP_FPS))

# Prepare the output video writer
output_video = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))

# Loop through each frame in the video
while True:
    ret, frame = video.read()
    if not ret:
        break  # Exit loop if there are no more frames

    # Convert the frame to grayscale for face detection
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    if len(faces) > 0:
        # Crop the first detected face
        x, y, w, h = faces[0]  # Assuming the first detected face is the one we want to crop
        face_frame = frame[y:y+h, x:x+w]

        # Resize the face to fit the original frame size (optional)
        resized_face_frame = cv2.resize(face_frame, (frame_width, frame_height))

        # Write the resized face frame to the output video
        output_video.write(resized_face_frame)
    else:
        # Write the original frame if no face is detected
        output_video.write(frame)

# Release the video objects
video.release()
output_video.release()

print("Video processing complete. The cropped video is saved as:", output_video_path)


Video processing complete. The cropped video is saved as: sample_output.mp4


In [ ]:
import cv2
import numpy as np
from skimage.metrics import structural_similarity as ssim

# Function to calculate PSNR
def calculate_psnr(original, processed):
    mse = np.mean((original - processed) ** 2)
    if mse == 0:
        return 100
    max_pixel = 255.0
    psnr_value = 20 * np.log10(max_pixel / np.sqrt(mse))
    return psnr_value



In [ ]:
import cv2
def calculate_ssim(original, processed):
    original_gray = cv2.cvtColor(original, cv2.COLOR_BGR2GRAY)
    processed_gray = cv2.cvtColor(processed, cv2.COLOR_BGR2GRAY)
    ssim_value, _ = ssim(original_gray, processed_gray, full=True)
    return ssim_value

In [ ]:
# Load the original and processed videos
original_video_path = "/content/D02_20240705162958 (online-video-cutter.com).mp4"
processed_video_path = "/content/D02_20240705162958 (online-video-cutter.com) output.mp4"

original_video = cv2.VideoCapture(original_video_path)
processed_video = cv2.VideoCapture(processed_video_path)

# Variables to accumulate PSNR and SSIM values
cumulative_psnr = 0
cumulative_ssim = 0
frame_count = 0

# Loop through each frame
while True:
    ret1, original_frame = original_video.read()
    ret2, processed_frame = processed_video.read()

    if not ret1 or not ret2:
        break  # Break if no more frames

    # Calculate PSNR and SSIM for each frame
    psnr_value = calculate_psnr(original_frame, processed_frame)
    ssim_value = calculate_ssim(original_frame, processed_frame)

    # Accumulate values
    cumulative_psnr += psnr_value
    cumulative_ssim += ssim_value
    frame_count += 1

# Calculate average PSNR and SSIM
average_psnr = cumulative_psnr / frame_count
average_ssim = cumulative_ssim / frame_count

print(f"Average PSNR: {average_psnr:.2f}")
print(f"Average SSIM: {average_ssim:.4f}")

# Release video objects
original_video.release()
processed_video.release()


Average PSNR: 36.12
Average SSIM: 0.8850
